<a href="https://colab.research.google.com/github/benson1231/DeepLearning/blob/main/ImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Classification

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# 加載 Fashion MNIST 資料集
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# 正規化並擴展維度,Fashion MNIST 資料集中，每個像素值原本的範圍是 0~255
x_train = x_train / 255.0
x_test = x_test / 255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

# 將標籤轉換為 one-hot,在深度學習中,標籤通常以one-hot向量的形式表示,這樣的格式適合用於計算交叉熵損失函數(categorical_crossentropy)
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# 資料增強
datagen = ImageDataGenerator(
    zoom_range=0.2,
    rotation_range=15,
    width_shift_range=0.05,
    height_shift_range=0.05
)
datagen.fit(x_train)

# 分割驗證集
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# 模型設計
model = Sequential([
    # 第一個卷積層：32個濾波器，大小為3x3，激活函數為ReLU，輸入形狀為28x28的單通道圖像(灰階)
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    # 最大池化層：將特徵圖縮小，池化區域大小為2x2
    MaxPooling2D(pool_size=(2, 2)),
    # 第二個卷積層：64個濾波器，大小為3x3，激活函數為ReLU
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    # 最大池化層：再次縮小特徵圖，池化區域大小為2x2
    MaxPooling2D(pool_size=(2, 2)),
    # 展平層：將多維特徵圖攤平成一維，以輸入全連接層
    Flatten(),
    # 全連接層：128個神經元，激活函數為ReLU，用於學習非線性特徵
    Dense(128, activation='relu'),
    # 輸出層：10個神經元（對應10個分類），激活函數為Softmax，輸出概率分布
    Dense(10, activation='softmax')
])


model.summary()

# 編譯模型
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 訓練模型
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=32),
    validation_data=(x_val, y_val),
    epochs=10,
    verbose=1
)

# 評估模型
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"\nTest Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 68s 44ms/step - accuracy: 0.6985 - loss: 0.8276 - val_accuracy: 0.8408 - val_loss: 0.4345
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 68s 45ms/step - accuracy: 0.8108 - loss: 0.4981 - val_accuracy: 0.8676 - val_loss: 0.3675
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 67s 44ms/step - accuracy: 0.8338 - loss: 0.4371 - val_accuracy: 0.8654 - val_loss: 0.3615
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 80s 43ms/step - accuracy: 0.8513 - loss: 0.3946 - val_accuracy: 0.8734 - val_loss: 0.3316
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 66s 44ms/step - accuracy: 0.8590 - loss: 0.3728 - val_accuracy: 0.8827 - val_loss: 0.3152
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 66s 44ms/step - accuracy: 0.8670 - loss: 0.3541 - val_accuracy: 0.8855 - val_loss: 0.3157
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 65s 43ms/step - accuracy: 0.8722 - loss: 0.3359 - val_accuracy: 0.8878 - val_loss: 0.3055
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 82s 43ms/step - accuracy: 0.8780 - loss: 0.32